In [2]:
import pickle
import time
import tqdm
import myslack

from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

#### - train.csv

In [3]:
%%time
train = pickle.load(open("./data/my_train.pkl", "rb"))
print("test data shape: "+str(train.shape))

test data shape: (95674, 5166)
CPU times: user 224 ms, sys: 782 ms, total: 1.01 s
Wall time: 1.53 s


In [4]:
train.shape

(95674, 5166)

#### - test.csv

In [5]:
# %%time
test = pickle.load(open("./data/my_test.pkl", "rb"))
print("test data shape: "+str(test.shape))

test data shape: (95674, 5165)


In [6]:
test.shape

(95674, 5165)

---

# LogisticRegression

### Train_X, Train_y

In [7]:
X = train.drop(columns='TripType')
y = train['TripType']

In [8]:
logistic_model = \
LogisticRegression(C=1.0, penalty='l2', solver='newton-cg',max_iter=150, multi_class='ovr', n_jobs=6).fit(X, y)

KeyboardInterrupt: 

In [ ]:
myslack.send_slack('finish')

### Confusion Matrix

In [ ]:
confusion_matrix(y, logistic_model.predict(X))

### Accuracy Score

In [ ]:
%time print('accuracy_score '+str(accuracy_score(y, logistic_model.predict(X))))

### Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

len(X_train), len(X_test), len(y_train), len(y_test)

In [ ]:
y_pred = logistic_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

### Prediction

In [57]:
%%time
result = logistic_model.predict_proba(test)

CPU times: user 3.93 s, sys: 2.01 s, total: 5.94 s
Wall time: 5.96 s


In [58]:
result.shape

(95674, 38)

In [59]:
result

array([[0.01079293, 0.00256098, 0.03336425, ..., 0.00856517, 0.00592726,
        0.07160228],
       [0.01929184, 0.00305447, 0.03874682, ..., 0.00827612, 0.00411306,
        0.1264491 ],
       [0.02443307, 0.00415962, 0.05648155, ..., 0.00625832, 0.00341404,
        0.17334964],
       ...,
       [0.02336697, 0.00431464, 0.05465828, ..., 0.00648695, 0.00363875,
        0.10935613],
       [0.00248126, 0.0010994 , 0.01552872, ..., 0.00849147, 0.06555776,
        0.02457799],
       [0.01580733, 0.00313748, 0.0407441 , ..., 0.0103657 , 0.00854993,
        0.078645  ]])

### Make Submission CSV file

In [60]:
samsub = pd.read_csv('./data/sample_submission.csv')
subform_df_columns = samsub.columns[1:]
result_df = pd.DataFrame(result)
result_df.columns = subform_df_columns
subform_df = pd.concat([test.reset_index()['VisitNumber'],result_df],axis=1)
subform_df.set_index('VisitNumber',inplace=True)
subform_df.tail()

,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,TripType_15,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
VisitNumber,,,,,,,,,,,,,,,,,,,,,
191338,0.013836,0.002298,0.031950,0.010132,0.040581,0.103614,0.076654,0.002480,0.000018,0.009091,...,0.032130,0.014220,0.046229,0.188676,0.043104,0.030123,0.020493,0.011359,0.010982,0.057030
191339,0.015951,0.002982,0.046431,0.010145,0.045984,0.135992,0.123180,0.001824,0.000039,0.018546,...,0.043825,0.012505,0.014955,0.074810,0.004769,0.006787,0.060124,0.038465,0.006433,0.071238
191340,0.023367,0.004315,0.054658,0.015162,0.051261,0.253784,0.142407,0.002239,0.000028,0.010075,...,0.023842,0.011191,0.015475,0.053945,0.004000,0.003881,0.013404,0.006487,0.003639,0.109356
191341,0.002481,0.001099,0.015529,0.003568,0.018923,0.041224,0.036097,0.002073,0.000014,0.005007,...,0.035167,0.033446,0.012338,0.212528,0.011859,0.008592,0.023422,0.008491,0.065558,0.024578
191348,0.015807,0.003137,0.040744,0.009217,0.041312,0.107490,0.103396,0.002457,0.000021,0.008457,...,0.031061,0.013418,0.016238,0.097377,0.004988,0.009582,0.025386,0.010366,0.008550,0.078645


In [61]:
subform_df.to_csv('./ExtraTree/EXT_est{}_dep{}.csv'.format(estimators,depth))